In [18]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os

In [19]:


# Get the current working directory
current_directory = os.getcwd()

# Set the current working directory to itself (no need to specify a path)
os.chdir(current_directory)

print(current_directory)

/Users/mmzo/Projects/COFFEE/jypter_notebook_coffee


Importing the dataset. All values are in KGs.

In [20]:
df_dom_consumption = pd.read_csv('data/Coffee_domestic_consumption.csv')
df_exports = pd.read_csv('data/Coffee_export.csv')
df_green_coffee_invent = pd.read_csv('data/Coffee_green_coffee_inventorie.csv')
df_import = pd.read_csv('data/Coffee_import.csv')
df_importers_consump = pd.read_csv('data/Coffee_importers_consumption.csv')
df_production = pd.read_csv('data/Coffee_production.csv')
df_reexport = pd.read_csv('data/Coffee_re_export.csv')

In [21]:
df_exports.head(10)

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_export
0,Angola,5040000,4260000,4800000,2340000,480000,2460000,3120000,3000000,3240000,...,480000,480000,360000,540000,660000,660000,540000,540000,1380000,43320000
1,Bolivia (Plurinational State of),9360000,4440000,5760000,2820000,5040000,5640000,7380000,6660000,5820000,...,4440000,3780000,3300000,3720000,1800000,1320000,1560000,1320000,1200000,137460000
2,Brazil,1016160000,1270980000,1127460000,1070280000,1036380000,868080000,915060000,1008060000,1088640000,...,2028360000,1712940000,1899060000,-2147483648,-2147483648,2056140000,1855500000,2138220000,-2147483648,33807709056
3,Burundi,35100000,41280000,38760000,25080000,30480000,31680000,13440000,31740000,22440000,...,13080000,23520000,11700000,15120000,13800000,12240000,10140000,12120000,17580000,646200000
4,Cameroon,156660000,105120000,98760000,42300000,32760000,24420000,33840000,82080000,44760000,...,29400000,37320000,16320000,22500000,23400000,16860000,14700000,17220000,15000000,1399920000
5,Central African Republic,11820000,8460000,6000000,8220000,8220000,13920000,5880000,12180000,6120000,...,4680000,4680000,60000,4500000,2580000,4800000,1080000,2340000,1140000,170700000
6,Colombia,836640000,755940000,993840000,814080000,706080000,588840000,635280000,655140000,675600000,...,464040000,430200000,580200000,657240000,762960000,769860000,779100000,768480000,820320000,19878480000
7,Congo,120000,60000,0,60000,60000,1140000,780000,180000,0,...,0,0,0,0,0,0,0,0,0,2400000
8,Costa Rica,135960000,144000000,145860000,141840000,126300000,124020000,145800000,125940000,122700000,...,74580000,82440000,80640000,72540000,67680000,60420000,59220000,72480000,62340000,2991840000
9,Côte d'Ivoire,256980000,228300000,272760000,241980000,146640000,149640000,165060000,282780000,261900000,...,46320000,102720000,117720000,89340000,85080000,85920000,51300000,91320000,126240000,4721640000


In [22]:
# Sort the DataFrame by 'Total Production' in descending order and select the top 10
top_exporters = df_exports.sort_values(by='Total_export', ascending=False).head(10)

#dropping the last column
top_exporters.drop(top_exporters.columns[-1], axis=1, inplace=True)

# setting data to long format
df_long = pd.melt(top_exporters, id_vars='Country', var_name='Year', value_name='Value')

df_long['Year'] = df_long['Year'].astype(int)



In [23]:
negative_entries = df_long[df_long['Value'] < 0]
negative_entries.head(100)

,Country,Year,Value
240,Brazil,2014,-2147483648
250,Brazil,2015,-2147483648
290,Brazil,2019,-2147483648


Incorrect values that need to be adjusted. Th3se are in thousand 60kg bags
2014: 37.335,1728254
2015: 37.562,8467468
2019: 40.697,8637087


In [24]:
values = [37335.1728254, 37562.8467468, 40697.8637087]

# Initialize an empty dictionary
replace_dict = {}

# Define the keys
keys = [2014, 2015, 2019]

# Loop through the list and create dictionary entries
for i, key in enumerate(keys):
    # Multiply the value by 60,000
    multiplied_value = values[i] * 60000
    
    # Add the value to the dictionary with the key
    replace_dict[key] = math.ceil(multiplied_value)

df_long.loc[(df_long['Country'] == 'Brazil') & (df_long['Year'].isin(replace_dict.keys())), 'Value'] = df_long['Year'].map(replace_dict)



Checking to see that this worked

In [25]:
df_long[(df_long['Country']=='Brazil') & (df_long['Year'].isin([2014,2015,2019]))]
# df_long[(df_long['Country']=='Brazil') & (df_long['Year']=='2014') ]
# df_long.dtypes

,Country,Year,Value
240,Brazil,2014,2240110370
250,Brazil,2015,2253770805
290,Brazil,2019,2441871823


It did! Now to make the values a little easier to read

In [26]:
# dividing by a million and rounding to two decimal places to make the values easier to read
df_long['Value'] = (df_long['Value'] / 1000000).round(2)

df_long.head(5)

,Country,Year,Value
0,Brazil,1990,1016.16
1,Viet Nam,1990,68.70
2,Colombia,1990,836.64
3,Indonesia,1990,414.18
4,India,1990,118.74


Now lets look at the world's biggest exporters of coffee!

In [27]:
# Sort the DataFrame by 'Country' and 'Year' to ensure data is in the correct order
df_long = df_long.sort_values(['Country', 'Year'])

# Calculate the moving average for each country
window_size = 10  # Adjust this for the desired smoothing effect
df_long['Smoothed_Value'] = df_long.groupby('Country')['Value'].rolling(window=window_size, center=False).mean().reset_index(level=0, drop=True).tolist()

# Create an interactive line plot
fig = px.line(df_long, x='Year', y='Value', color='Country', title='Coffee Exports Over Time')


# Add hover data (displays the value when hovering over a point)
fig.update_traces(mode='markers+lines', hovertemplate='%{y:.2f}')

# Customize the plot
fig.update_xaxes(title_text='Year',tickmode='linear', tickvals=df_long['Year'].unique(),tickangle=45)
fig.update_yaxes(title_text='Million Kgs of Coffee Exported')

# Show the plot
fig.show()

Based on this graph we can see that Brazil is the biggest coffee exporter, followed by Vietnam and then later by Colombia.

A couple of interesting things stand out straight away here: 
    1. Brazil enjoyed a period of massive growth from 2017 to 2019.
    2. Vietnam sprinted out of the gate and in 2000 took over from Cambodia to become the second biggest exporter. They've been unstoppable ever since. Can they overtake Brazil? ...
    3. Probably not as the export cycle for Vietnam seems to be extremely volatile and is characterised by boom/bust, particulary over the last decade.

That's pretty cool to know! But before I move on, I am a little curious about Colombia there: there seems to be an uptick over the last five years, but overall are they still decreasing? Let's apply a rolling average of 5 years to smooth out the data and check the general trend:

In [28]:
# Calculate the moving average for each country
window_size = 5  # Adjust this for the desired smoothing effect
df_long['Smoothed_Value'] = df_long.groupby('Country')['Value'].rolling(window=window_size, center=False).mean().reset_index(level=0, drop=True).tolist()

# Create an interactive line plot
fig = px.line(df_long, x='Year', y='Smoothed_Value', color='Country', title='Coffee Exports Over Time')

# Add hover data (displays the value when hovering over a point)
fig.update_traces(mode='markers+lines', hovertemplate='%{y:.2f}')

# Customize the plot
fig.update_xaxes(title_text='Year',tickmode='linear', tickvals=df_long['Year'].unique(),tickangle=45)
fig.update_yaxes(title_text='Million Kgs of Coffee Exported')



fig.show()

And, yes, that does make things a bit clearer. 

As we saw above Colombia was decreasing its exports gradually over the years up until 2013. However, that uptick that we saw above is also evident here. Can they regain some of their former export power?

Another interesting thing I notice here is how Indonesia stands out from the pack as the fourth biggest exporter on average. But it looks like in recent years it is getting a run for its money from the other exporters in the top 10 pack!

NOW LETS TRY THE SAME WITH IMPORTS

In [29]:
df_import.head(5)

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_import
0,Austria,112800000,123480000,132360000,110160000,85020000,73860000,72600000,77640000,77580000,...,87120000,93540000,93300000,91500000,90780000,89700000,87600000,85740000,86880000,2765760000
1,Belgium,0,0,0,0,0,0,0,0,0,...,349680000,340080000,330120000,312720000,332040000,363120000,340620000,344520000,371940000,6240540000
2,Belgium/Luxembourg,120900000,104760000,109680000,123780000,135720000,144060000,151560000,152640000,214800000,...,0,0,0,0,0,0,0,0,0,1257900000
3,Bulgaria,16080000,12000000,10920000,23820000,27780000,30900000,16320000,17640000,20520000,...,28920000,33600000,36540000,37260000,40920000,46740000,42420000,44220000,47100000,830700000
4,Croatia,0,0,10080000,9780000,11580000,19200000,19320000,23100000,20340000,...,23460000,23040000,24780000,25200000,26220000,28740000,26340000,26760000,27960000,622080000


Hmm that's a lot of European countriese in the top 5 rows, I wonder actually which countries make up this list. Lets check it out:

In [30]:
unique_countries = sorted(df_import['Country'].unique())

for country in unique_countries:
    print(country)


   Austria
   Belgium
   Belgium/Luxembourg
   Bulgaria
   Croatia
   Cyprus
   Czechia
   Denmark
   Estonia
   Finland
   France
   Germany
   Greece
   Hungary
   Ireland
   Italy
   Latvia
   Lithuania
   Luxembourg
   Malta
   Netherlands
   Poland
   Portugal
   Romania
   Slovakia
   Slovenia
   Spain
   Sweden
Japan
Norway
Russian Federation
Switzerland
Tunisia
United Kingdom
United States of America


Okay! Well for my money I'd say that most of those countries are in the EU, and since it is a trading bloc with a bit of clout in the world economy it might make sense to take a look at it as a separate entity. 

So lets add up all the EU countries and include it as a country in our dataset!

First, however, we need to put the data into long format so we can graph it later.

In [31]:
# Sort the DataFrame by 'Total Production' in descending order and select the top 5 and the EU
top_importers = df_import.sort_values(by='Total_import', ascending=False)

#dropping the last column
top_importers.drop(top_importers.columns[-1], axis=1, inplace=True)

# setting data to long format
imports_long = pd.melt(top_importers, id_vars='Country', var_name='Year', value_name='Value')

imports_long['Year'] = imports_long['Year'].astype(int)

#to convert to millions of kgs (making it more readable)
imports_long['Value'] = (imports_long['Value']/1000000).round(2)

#checking everything worked
imports_long.head(5)

,Country,Year,Value
0,United States of America,1990,1260.42
1,Germany,1990,820.26
2,Italy,1990,314.52
3,Japan,1990,319.80
4,France,1990,378.06


And now that's done we summarise the EU values.

Its neccessary to first filter out all the EU nations that I've included in the list (copy and paste from the one above).

Then I group by the year and sum up all the values for each year. I create a new dataframe and add the Country name EU, and then finally concat the original dataframe with that of new one.

In [32]:
# List of EU nations
eu_countries = [
    'Austria', 'Belgium', 'Belgium/Luxembourg', 'Bulgaria', 'Croatia', 'Cyprus',
    'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
    'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
    'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain'
]

# Filter the DataFrame to include only EU nations
eu_imports_df = imports_long[imports_long['Country'].str.strip().isin(eu_countries)]

# Group by year and calculate the total imports for each year
eu_total_imports = eu_imports_df.groupby('Year')['Value'].sum().reset_index()

# Create a new DataFrame with 'EU' as the country name for each year
eu_combined_df = pd.DataFrame({'Year': eu_total_imports['Year'], 'Country': 'EU', 'Value': eu_total_imports['Value']})

# Concatenate the new 'EU' DataFrame with the original DataFrame
combined_imports = pd.concat([imports_long, eu_combined_df], ignore_index=True)

# Print the updated DataFrame with 'EU' included
print(combined_imports)



                       Country  Year    Value
0     United States of America  1990  1260.42
1                      Germany  1990   820.26
2                        Italy  1990   314.52
3                        Japan  1990   319.80
4                       France  1990   378.06
...                        ...   ...      ...
1075                        EU  2015  4192.56
1076                        EU  2016  4443.06
1077                        EU  2017  4314.90
1078                        EU  2018  4553.22
1079                        EU  2019  4662.54

[1080 rows x 3 columns]


After that I use the original dataset to highlight the top five coffee producing countries. I've also included the EU bloc as a whole for interest.

In [33]:
# Assuming you have a DataFrame named 'df' with columns 'Country', 'Year', and 'Value'

# Calculate the total imports for each country
country_imports = combined_imports.groupby('Country')['Value'].sum().reset_index()

# Sort the countries by total imports in descending order
sorted_countries = country_imports.sort_values(by='Value', ascending=False)

# Select the top 6 countries
top_6_importers = sorted_countries.head(6)

# Create a list of the top 6 country names
top_6_countries = top_6_importers['Country'].tolist()

# Filter the original DataFrame to include only data for the top 6 countries
filtered_df = combined_imports[combined_imports['Country'].isin(top_6_countries)].copy()


Okay with that done, let's check out the results of this!

In [34]:
# Create an interactive line plot
importers_fig = px.line(filtered_df, x='Year', y='Value', color='Country', title='Coffee Imports Over Time')

# Add hover data (displays the value when hovering over a point)
importers_fig.update_traces(mode='markers+lines', hovertemplate='%{y:.2f}')

# Customize the plot
importers_fig.update_xaxes(title_text='Year',tickmode='linear', tickvals=df_long['Year'].unique(),tickangle=45)
importers_fig.update_yaxes(title_text='Million Kgs of Coffee Imported')

# Show the plot
importers_fig.show()

And well, that does show pretty conclusively that the EU trading bloc is a pretty powerful force in the global coffee market. 

Just looking at the countries by themselves though, it seems that the US has a pretty dominant position. However, perhaps surprisingly it is Germany who takes second place. The other three countries far beliow, Italy, Japan and France, illustrate that, well, coffee seems to be a drink of choice for rich, developed nations. 